In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 40.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=473b5ffeb0a8537bdf655cce8f20a04385a3518ebcf3c3a19184f86adb2fc30e
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

spark = SparkSession.builder.getOrCreate()

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/BigData_Dataset/paysim.csv",header=True,inferSchema=True)

In [ ]:
df.columns

['step',
 'type',
 'amount',
 'nameOrig',
 'oldbalanceOrg',
 'newbalanceOrig',
 'nameDest',
 'oldbalanceDest',
 'newbalanceDest',
 'isFraud',
 'isFlaggedFraud']

In [ ]:
df = df.drop("isFraud","isFlaggedFraud")

In [ ]:
df.show()

+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|   1| PAYMENT|  9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|
|   1| PAYMENT|  1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|
|   1|TRANSFER|    181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|
|   1|CASH_OUT|    181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|
|   1| PAYMENT| 11668.14|C2048537720|      41554.0|      29885.86|M1230701703|           0.0|           0.0|
|   1| PAYMENT|  7817.71|  C90045638|      53860.0|      46042.29| M573487274|           0.0|           0.0|
|   1| PAYMENT|  71

In [ ]:
df.groupBy("step").count().show()

+----+-----+
|step|count|
+----+-----+
|   1| 2708|
|   6| 1660|
|   3|  552|
|   5|  665|
|   4|  565|
|   7| 2835|
|   2| 1014|
+----+-----+



In [ ]:
steps = df.select("step").distinct().collect()
for step in steps[:]:
  _df = df.where(f"step = {step[0]}")
  _df.coalesce(1).write.mode("append").option("header","true").csv("/content/drive/MyDrive/BigData_Dataset/Output")

In [ ]:
!cd /content/drive/MyDrive/BigData_Dataset/Output && ls

part-00000-1481adc1-102f-4a20-a03b-beaebdd2cce0-c000.csv
part-00000-216e2e24-6880-43b5-a151-a6fffb4ac366-c000.csv
part-00000-229b28ea-3b4d-4d63-87a8-9e7632503360-c000.csv
part-00000-32e038b7-eff9-4c56-ac81-3070a5df6d65-c000.csv
part-00000-33131987-4fc1-4df4-be9c-37875d38a54e-c000.csv
part-00000-35b211e2-2341-4b77-a505-c589e64303ce-c000.csv
part-00000-4466c461-0492-4911-92f0-588700388eef-c000.csv
part-00000-4a6383d2-708f-48a9-98d1-b238b0095ac8-c000.csv
part-00000-5345870c-9b71-480f-b225-0062ea8f42f4-c000.csv
part-00000-5ac13b82-9fa1-4dfd-aac3-d58dd08d4ff8-c000.csv
part-00000-6b6ae93f-08e1-476b-a065-83042a446ee5-c000.csv
part-00000-787e7778-1020-4aac-b09a-b585c6e23924-c000.csv
part-00000-9dedbc02-914f-4ab4-93ed-15785c8bf8ac-c000.csv
part-00000-b2600c83-d383-4796-9db5-444afef1aa58-c000.csv
part-00000-c248c37b-8f19-4b95-a1c2-22dcb69bf6f5-c000.csv
part-00000-e7be80c4-cf0d-4bc0-b71d-f0e98a04c4e0-c000.csv
part-00000-ec30c218-34f1-4045-8a7b-cb8d003b2901-c000.csv
part-00000-f17b32f3-d909-4c19-b

In [ ]:
part = spark.read.csv("/content/drive/MyDrive/BigData_Dataset/Output/part-00000-1481adc1-102f-4a20-a03b-beaebdd2cce0-c000.csv",
                      header=True,
                      inferSchema=True)

In [ ]:
part.groupBy("step").count().show()

+----+-----+
|step|count|
+----+-----+
|   3|  552|
+----+-----+



In [ ]:
dataSchema = part.schema

dataSchema

StructType(List(StructField(step,IntegerType,true),StructField(type,StringType,true),StructField(amount,DoubleType,true),StructField(nameOrig,StringType,true),StructField(oldbalanceOrg,DoubleType,true),StructField(newbalanceOrig,DoubleType,true),StructField(nameDest,StringType,true),StructField(oldbalanceDest,DoubleType,true),StructField(newbalanceDest,DoubleType,true)))

In [ ]:
streaming = (
  spark.readStream.schema(dataSchema)
  .option("maxFilesPerTrigger",1)
  .csv("/content/drive/MyDrive/BigData_Dataset/Output/")
)

In [ ]:
dest_count = streaming.groupBy("nameDest").count().orderBy(F.desc("count"))

In [ ]:
activityQuery = (
    dest_count.writeStream.queryName("dest_counts")
    .format("memory")
    .outputMode("complete")
    .start()
)

import time

for x in range(20):
    _df = spark.sql(
        "SELECT * FROM dest_counts WHERE nameDest != 'nameDest' AND count >= 2"
    )
    if _df.count() > 0:
        _df.show(10)
    time.sleep(0.5)

+-----------+-----+
|   nameDest|count|
+-----------+-----+
| C985934102|   32|
|C1590550415|   32|
| C564160838|   28|
| C401424608|   22|
|  C33524623|   22|
|C1023714065|   22|
|C1286084959|   21|
|C1789550256|   20|
| C998351292|   20|
|C2083562754|   20|
+-----------+-----+
only showing top 10 rows

+-----------+-----+
|   nameDest|count|
+-----------+-----+
| C985934102|   32|
|C1590550415|   32|
| C564160838|   28|
| C401424608|   22|
|  C33524623|   22|
|C1023714065|   22|
|C1286084959|   21|
|C1789550256|   20|
| C998351292|   20|
|C2083562754|   20|
+-----------+-----+
only showing top 10 rows

+-----------+-----+
|   nameDest|count|
+-----------+-----+
| C985934102|   32|
|C1590550415|   32|
| C564160838|   28|
| C401424608|   22|
|  C33524623|   22|
|C1023714065|   22|
|C1286084959|   21|
|C1789550256|   20|
| C998351292|   20|
|C2083562754|   20|
+-----------+-----+
only showing top 10 rows

+-----------+-----+
|   nameDest|count|
+-----------+-----+
| C985934102|   42|
|C

In [ ]:
spark.streams.active[0].isActive

True

In [ ]:
activityQuery.status

{'isDataAvailable': False,
 'isTriggerActive': True,
 'message': 'Getting offsets from FileStreamSource[file:/content/drive/MyDrive/BigData_Dataset/Output]'}

In [ ]:
activityQuery.stop()